# QTM350 - Group Project
## Group members

Please find the names of the group members below:

In [1]:
# Import packages
import sqlite3
import pandas as pd
import random

In [2]:
# Connect to the database
connection = sqlite3.connect("groups.db")
cursor = connection.cursor()

In [3]:
# Execute the SQL commands to create the table
cursor.execute('''
DROP TABLE IF EXISTS groups;
''')

cursor.execute('''
CREATE TABLE groups (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT NOT NULL UNIQUE,
    emory_id INT UNIQUE, -- Allow NULL and ensure uniqueness
    group_id INT
);
''')

# Commit the changes for table creation
connection.commit()

In [4]:
# Insert data into the table
insert_sql = '''
INSERT INTO groups (student_name, emory_id, group_id) VALUES (?, ?, ?)
'''

# List of tuples: (student_name, emory_id, group_id)
# Using Python None here, as executemany handles it correctly for sqlite3
students_data = [
    ('Jacky An', 2486597, 1),
    ('Anushka Basu', 2551669, 4),
    ('Manny Bettencourt', 2515917, 6),
    ('Kush Bhatia', 2492303, 6),
    ('Davis Boor', 2556176, None),
    ('Joyce Chen', 2485906, None),
    ('Nicole Chen', 2608628, 8),
    ('Yunqing Ivy Chen', 2585468, 7),
    ('Anna Choi', 2452634, 2),
    ('Daisy Cossio', 2503179, 5),
    ('Jackson Fang', 2432239, 7),
    ('Lucas Goldfein', 2486286, None),
    ('Felix Huang', 2517419, 7),
    ('Sean Jeon', 2550044, 2),
    ('Emma Jinright', 2488724, None),
    ('Minjoo Kim', 2580377, 2),
    ('Jooha Lee', 2433418, 2),
    ('Krystal Li', 2587957, 3),
    ('Zihan Liang', 2609381, 8),
    ('Amber Lin', 2486031, 3),
    ('Lucas Lobo', 2555247, None),
    ('Katherine Martini', 2549333, None),
    ('Liane Muir', 2493825, 5),
    ('Nora Ni', 2599639, 3),
    ('Daniel Nickas', 2549711, None),
    ('Kei Nie', 2541685, 7),
    ('Phoebe Pan', 2630423, 8),
    ('Adam Pastor', 2565464, None),
    ('Aryan Patel', 2494029, 6),
    ('Nick Richards', 2495699, None),
    ('Caleb Sharkey', 2514723, None),
    ('Iliyan Sherali', 2520396, 5),
    ('Saanvi Sood', 2494531, 4),
    ('Brian Tekadtuera', 2577545, 4),
    ('Mingke Tian', 2636361, 8),
    ('Rahul Ulman', 2434956, None),
    ('Aanya Vusirikala', 2558223, 4),
    ('Xinyi Wang', 2549813, 3),
    ('Iris Wu', 2515186, 3),
    ('Bernice Yuan', 2487091, 1),
    ('Eric Zou', 2548644, 1)
]

cursor.executemany(insert_sql, students_data)

# Commit the transaction for data insertion
connection.commit()

In [5]:
# --- Random Group Assignment Logic ---
print("\n--- Assigning Remaining Students to Groups ---")

# Ensure connection is still open or reconnect
try:
    # Check if cursor is usable by executing a simple query
    cursor.execute("SELECT 1") 
except (sqlite3.ProgrammingError, AttributeError, NameError): 
    print("Reconnecting to database for group assignment...")
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

# Fetch unassigned students (student_id)
cursor.execute("SELECT student_id FROM groups WHERE group_id IS NULL")
unassigned_rows = cursor.fetchall()
unassigned_ids = [row[0] for row in unassigned_rows] # Extract IDs
n = len(unassigned_ids)

print(f"Found {n} unassigned students.")

g4 = 0 # Number of groups of 4
g3 = 0 # Number of groups of 3
possible = True

if n >= 3:
    # Calculate number of groups of 4 and 3, prioritising groups of 4
    temp_g4 = n // 4
    remainder = n % 4 # Remainder after forming groups of 4

    # Adjust to make remainder divisible by 3
    while remainder % 3 != 0:
        if temp_g4 == 0: # Cannot form any more groups of 4
            possible = False
            print(f"Cannot divide {n} students perfectly into groups of 3 or 4.")
            break
        temp_g4 -= 1    # Reduce one group of 4
        remainder += 4  # Add those 4 students back to the remainder pool

    if possible:
        g4 = temp_g4
        g3 = remainder // 3
        print(f"Creating {g4} groups of 4 and {g3} groups of 3.")

        # Shuffle the student IDs randomly
        random.shuffle(unassigned_ids)

        # Find the next available group ID
        cursor.execute("SELECT MAX(group_id) FROM groups WHERE group_id IS NOT NULL")
        max_existing_group_id_result = cursor.fetchone()
        # Handle case where there are no existing groups (max_existing_group_id_result[0] would be None)
        max_existing_group_id = max_existing_group_id_result[0] if max_existing_group_id_result and max_existing_group_id_result[0] is not None else 0
        next_group_id = max_existing_group_id + 1 
        print(f"Starting new groups from ID: {next_group_id}")

        current_student_index = 0
        update_sql = "UPDATE groups SET group_id = ? WHERE student_id = ?"

        # Assign groups of 4
        print("\nAssigning groups of 4...")
        for i in range(g4):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 4]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 4
            else:
                print("  Warning: Tried to assign an empty group of 4.")

        # Assign groups of 3
        print("\nAssigning groups of 3...")
        for i in range(g3):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 3]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 3
            else:
                 print("  Warning: Tried to assign an empty group of 3.")

        # Commit the updates for group assignments
        connection.commit()
        print("\nFinished assigning groups.")

else:
    print("Not enough unassigned students (need at least 3) to form new groups.")

# The final query cell will display the results.
print("--- End of Group Assignment ---\n")


--- Assigning Remaining Students to Groups ---
Found 11 unassigned students.
Creating 2 groups of 4 and 1 groups of 3.
Starting new groups from ID: 9

Assigning groups of 4...
  Assigning group 9 to students (IDs): [25, 30, 15, 5]
  Assigning group 10 to students (IDs): [31, 22, 21, 6]

Assigning groups of 3...
  Assigning group 11 to students (IDs): [12, 28, 36]

Finished assigning groups.
--- End of Group Assignment ---



In [6]:
# Query the table and display results using pandas
query = '''
SELECT * FROM groups
ORDER BY group_id, student_name
'''
# Reconnect if connection was closed, or ensure it's open
connection = sqlite3.connect("groups.db")
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

# Display the DataFrame
df

student_id       student_name  emory_id  group_id
0           40       Bernice Yuan   2487091         1
1           41           Eric Zou   2548644         1
2            1           Jacky An   2486597         1
3            9          Anna Choi   2452634         2
4           17          Jooha Lee   2433418         2
5           16         Minjoo Kim   2580377         2
6           14          Sean Jeon   2550044         2
7           20          Amber Lin   2486031         3
8           39            Iris Wu   2515186         3
9           18         Krystal Li   2587957         3
10          24            Nora Ni   2599639         3
11          38         Xinyi Wang   2549813         3
12          37   Aanya Vusirikala   2558223         4
13           2       Anushka Basu   2551669         4
14          34   Brian Tekadtuera   2577545         4
15          33        Saanvi Sood   2494531         4
16          10       Daisy Cossio   2503179         5
17          32     Iliyan Sherali   2520396         5
18          23         Liane Muir   2493825         5
19          29        Aryan Patel   2494029         6
20           4        Kush Bhatia   2492303         6
21           3  Manny Bettencourt   2515917         6
22          13        Felix Huang   2517419         7
23          11       Jackson Fang   2432239         7
24          26            Kei Nie   2541685         7
25           8   Yunqing Ivy Chen   2585468         7
26          35        Mingke Tian   2636361         8
27           7        Nicole Chen   2608628         8
28          27         Phoebe Pan   2630423         8
29          19        Zihan Liang   2609381         8
30          25      Daniel Nickas   2549711         9
31           5         Davis Boor   2556176         9
32          15      Emma Jinright   2488724         9
33          30      Nick Richards   2495699         9
34          31      Caleb Sharkey   2514723        10
35           6         Joyce Chen   2485906        10
36          22  Katherine Martini   2549333        10
37          21         Lucas Lobo   2555247        10
38          28        Adam Pastor   2565464        11
39          12     Lucas Goldfein   2486286        11
40          36        Rahul Ulman   2434956        11